In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import json
import numpy as np

In [2]:
peptides = []
fname = "../data/XMAn/pr5004467_si_001.txt"
ix = 0

cancer_proteome_ref = {}
with open(fname, "r") as f:
    label=None
    prot_seq = ""
    for line in tqdm(f):
        if line.startswith(">"):
            if label is not None:
                cancer_proteome_ref[label+"_"+str(ix)] = prot_seq
                ix += 1
                prot_seq = ""
            label = line.strip()
            continue
        else:
            prot_seq += line.strip()
            
    cancer_proteome_ref[label] = prot_seq
print(len(cancer_proteome_ref))

0it [00:00, ?it/s]

872125


In [3]:
def extract_peptides_from_protein_sequence(protein_seq, l=9):
    if len(protein_seq)<l:
        return []
    peptides = []
    for i in range(len(protein_seq)-l+1):
        peptides.append(protein_seq[i:i+l])
    return peptides

In [4]:
human_cancer_peptides = []
for l in tqdm(range(8, 13)):
    for prot_seq in tqdm(list(cancer_proteome_ref.values())):
        peptides = extract_peptides_from_prot(prot_seq, l)
        human_cancer_peptides.extend(peptides)
len(human_cancer_peptides)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/872125 [00:00<?, ?it/s]

  0%|          | 0/872125 [00:00<?, ?it/s]

  0%|          | 0/872125 [00:00<?, ?it/s]

  0%|          | 0/872125 [00:00<?, ?it/s]

  0%|          | 0/872125 [00:00<?, ?it/s]

208884438

In [5]:
with open("../processed_data/UniProt/human_peptides.json", "r") as f:
    reference_human_peptides = json.load(f)
reference_human_peptides = set(reference_human_peptides) 
len(reference_human_peptides)

52208587

In [6]:
filtered_cancer_peptides = []
for p in tqdm(human_cancer_peptides):
    if p not in reference_human_peptides:
        filtered_cancer_peptides.append(p)
len(filtered_cancer_peptides)

  0%|          | 0/208884438 [00:00<?, ?it/s]

42498820

In [7]:
filtered_cancer_peptides_set = set(filtered_cancer_peptides)
len(filtered_cancer_peptides_set)

39702919

In [8]:
with open("../processed_data/XMAn_filtered_peptides.json", "w") as f:
    json.dump(list(filtered_cancer_peptides_set), f, indent=2)